# Software Pricing
## Data Cleaning
**Paul Sims**

# Setup

## Imports

In [1]:
# Plotting 
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")
sns.set(font_scale = 1.5)

# Data manipulation
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Utility
import rootpath as rp

## Read in data

In [2]:
# Set paths

# Root
rpath = rp.detect() + '/'

# Data directory
data_direct = 'data/'

In [3]:
# Read in data

df_test_res = pd.read_csv(rpath+data_direct+'raw/'+'test_results.csv',
                          sep=",", index_col=0)
df_users = pd.read_csv(rpath+data_direct+'raw/'+'user_table.csv', sep=",")

# Data Cleaning

## Test results

### Data summaries

In [4]:
df_test_res.shape

(316800, 8)

In [5]:
df_test_res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 316800 entries, 0 to 316799
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           316800 non-null  int64  
 1   timestamp         316466 non-null  object 
 2   source            316488 non-null  object 
 3   device            316800 non-null  object 
 4   operative_system  296461 non-null  object 
 5   test              316800 non-null  int64  
 6   price             316800 non-null  float64
 7   converted         316800 non-null  int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 21.8+ MB


- Missing values in timestamp, source, and os system
- Objects may need to be one hot encoded

In [6]:
# Rename operative_system 

df_test_res.rename(columns={"operative_system":"os"}, inplace=True)

In [7]:
# Count NA values

df_test_missing_cts_df = pd.DataFrame({#'column_name': df_test_res.columns,
                                 'missing cts': df_test_res.isnull().sum()})
print('Missing value counts')
df_test_missing_cts_df.sort_values(by=['missing cts'], ascending=False)

Missing value counts


,missing cts
os,20339
timestamp,334
source,312
user_id,0
device,0
test,0
price,0
converted,0


In [8]:
# Check proportion of data that is missing

print('Percent missing values')

df_test_missing_perc = pd.DataFrame({'missing cts': 
                                round((df_test_res.isnull().sum() / 
                                       len(df_test_res)) * 100,2)})
df_test_missing_perc.sort_values(by=['missing cts'], ascending=False)

Percent missing values


,missing cts
os,6.42
timestamp,0.11
source,0.10
user_id,0.00
device,0.00
test,0.00
price,0.00
converted,0.00


- OS has the most and might benefit from imputation
- The percentage of missing values is a bit small for the others and might be ok to drop those rows
- 

In [9]:
# Check how NA values in OS are distributed with conversion and test

df_miss_os = df_test_res[df_test_res['os'].isnull()]

In [10]:
df_miss_os.shape

(20339, 8)

In [11]:
# Print total conversion counts for both tests (including rows with NAs)

df_con_tot = df_test_res.groupby(['test','converted'], dropna=False) \
                                ['converted'].count()
df_con_tot

test  converted
0     0            198692
      1              4035
1     0            112300
      1              1773
Name: converted, dtype: int64

In [12]:
# Print the proportion of conversion counts by test that are NA for OS

df_miss_os.groupby(['test','converted'], dropna=False)['converted'].count()/ \
                    df_con_tot

test  converted
0     0            0.063032
      1            0.044857
1     0            0.067418
      1            0.035533
Name: converted, dtype: float64

- Very small percentages and equal in both groups so removal would have a minimal and similar effect on analysis of the test effect

In [13]:
# Get unique values for columns besides date

# List of columns to obtain unique values
col_to_check = ['source','device','os','test','price','converted']

for col in col_to_check:
    print(col)
    print(df_test_res[col].unique())
    print('')

source
['ads_facebook' 'seo-google' 'ads-bing' 'direct_traffic' 'ads-google'
 'seo_facebook' 'friend_referral' 'ads_other' 'ads-yahoo' 'seo-yahoo'
 'seo-bing' 'seo-other' nan]

device
['mobile' 'web']

os
['iOS' 'android' 'mac' 'windows' nan 'i0S' 'wnddows' 'anroid' 'apple']

test
[0 1]

price
[3.90000000e+01 5.90000000e+01 7.60350245e+08 ... 8.08182142e+08
 6.55179349e+08 6.21407779e+08]

converted
[0 1]



- Source could be combined and split based on the type of source and the specific source
- Some overlap with OS that could be changed
- Errors with price, more than two values

In [14]:
# Create source type column and populate based on
# ads, seo, friend_referral, or direct traffic

source_type_conditions = [
    (df_test_res['source'].str.contains('ads') == True),
    (df_test_res['source'].str.contains('seo') == True),
    (df_test_res['source'].str.contains('direct_traffic') == True),
    (df_test_res['source'].str.contains('friend_referral') == True),
]

source_type_values = ['ads', 'seo', 'direct_traffic', 'friend_referral']

df_test_res['source_type'] = np.select(source_type_conditions,
                                       source_type_values)

In [15]:
# Remove ads and SEO from source

replacement = {
    "ads_|ads-": "",
    "seo_|seo-": ""
}

df_test_res['source'] = df_test_res['source'].replace(replacement, regex=True)

In [16]:
# Consolidate OS categories to apple, windows, and android

os_replacement = {
    "iOS|mac|i0S": "apple",
    "wnddows": "windows",
    "anroid":"android"
}

df_test_res['os'] = df_test_res['os'].replace(os_replacement, regex=True)

In [17]:
# Fix price to match test

price_conditions = [
    (df_test_res['test'] == 0),
    (df_test_res['test'] == 1)
]
 
price_values = [39.00, 59.00]

df_test_res['price'] = np.select(price_conditions,
                                       price_values)

In [18]:
df_test_res.head()

,user_id,timestamp,source,device,os,test,price,converted,source_type
0,604839,2015-05-08 03:38:34,facebook,mobile,apple,0,39.0,0,ads
1,624057,2015-05-10 21:08:46,google,mobile,android,0,39.0,0,seo
2,317970,2015-04-04 15:01:23,bing,mobile,android,0,39.0,0,ads
3,685636,2015-05-07 07:26:01,direct_traffic,mobile,apple,1,59.0,0,direct_traffic
4,820854,2015-05-24 11:04:40,facebook,web,apple,0,39.0,0,ads


In [19]:
# Check summary statistics for non-objects

df_test_res.describe()

,user_id,test,price,converted
count,316800.000000,316800.000000,316800.000000,316800.000000
mean,499281.341840,0.360079,46.201578,0.018333
std,288591.154044,0.480024,9.600475,0.134154
min,3.000000,0.000000,39.000000,0.000000
25%,249525.750000,0.000000,39.000000,0.000000
50%,499021.500000,0.000000,39.000000,0.000000
75%,749025.500000,1.000000,59.000000,0.000000
max,1000000.000000,1.000000,59.000000,1.000000


In [20]:
# Check screenshot of data

df_test_res.head()

,user_id,timestamp,source,device,os,test,price,converted,source_type
0,604839,2015-05-08 03:38:34,facebook,mobile,apple,0,39.0,0,ads
1,624057,2015-05-10 21:08:46,google,mobile,android,0,39.0,0,seo
2,317970,2015-04-04 15:01:23,bing,mobile,android,0,39.0,0,ads
3,685636,2015-05-07 07:26:01,direct_traffic,mobile,apple,1,59.0,0,direct_traffic
4,820854,2015-05-24 11:04:40,facebook,web,apple,0,39.0,0,ads


- Timestamp has both date and time
- Need to separate into date and time columns

In [21]:
# Separate timestamp into respective dates and times

# Convert timestamp to a temp datetime format and coerce errors into NaT
df_test_res['temp_dt'] = pd.to_datetime(df_test_res['timestamp'],
                                        errors='coerce')

In [22]:
df_test_res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 316800 entries, 0 to 316799
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   user_id      316800 non-null  int64         
 1   timestamp    316466 non-null  object        
 2   source       316488 non-null  object        
 3   device       316800 non-null  object        
 4   os           296461 non-null  object        
 5   test         316800 non-null  int64         
 6   price        316800 non-null  float64       
 7   converted    316800 non-null  int64         
 8   source_type  316800 non-null  object        
 9   temp_dt      305905 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 26.6+ MB


In [23]:
# Check source of NaT errors

df_test_res[df_test_res['temp_dt'].isna()][['timestamp','temp_dt']][:10]

,timestamp,temp_dt
54,2015-04-24 12:60:46,NaT
104,2015-04-24 11:60:20,NaT
121,2015-03-14 12:60:02,NaT
278,2015-04-04 02:23:60,NaT
282,2015-05-11 12:60:53,NaT
287,2015-04-26 11:60:44,NaT
330,2015-03-20 12:20:60,NaT
361,2015-04-10 07:42:60,NaT
383,2015-03-10 12:31:60,NaT
388,2815-04-11 11:23:54,NaT


- Note impossible presence of 60 values in minutes and seconds
- Need to convert these to 59
- Also note impossible year, e.g. 2815
- Need to convert the 8 to a 0

In [24]:
# Convert 60s to 59s in time

df_test_res['temp_timestamp'] = df_test_res['timestamp'].str.replace('60', '59')

In [25]:
# Convert 2815 to 2015

df_test_res['temp_timestamp'] = (
    df_test_res['temp_timestamp'].str.replace('2815','2015')
    )

In [26]:
# Convert back to datetime

df_test_res['temp_dt2'] = pd.to_datetime(df_test_res['temp_timestamp'],
                                        errors='coerce')

In [27]:
# Check for errors converting to timestamp

df_test_res[df_test_res['temp_dt2'].isna()][['timestamp','temp_dt2']][:10]

,timestamp,temp_dt2
586,NaN,NaT
986,NaN,NaT
2285,NaN,NaT
2397,NaN,NaT
2937,NaN,NaT
3610,NaN,NaT
3624,NaN,NaT
4926,NaN,NaT
6521,NaN,NaT
6678,NaN,NaT


In [28]:
# Convert corrected timestamp to datetime format

df_test_res['timestamp'] = pd.to_datetime(df_test_res['temp_timestamp'],
                                        errors='coerce')

In [29]:
# Remove placeholder check variables

df_test_res.drop(['temp_dt','temp_timestamp','temp_dt2'],
                 axis=1,
                 inplace=True)

In [30]:
# Create date and time columns

df_test_res['date'] = df_test_res['timestamp'].dt.date
df_test_res['time'] = df_test_res['timestamp'].dt.time

In [31]:
# Check for duplicate user ids
# This would indicate if we have the same user multiple times
# If True, we have all unique ids, if false, there are duplicates

df_test_res['user_id'].nunique() == len(df_test_res)

True

## User data

In [32]:
df_users.shape

(275616, 5)

In [33]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275616 entries, 0 to 275615
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  275616 non-null  int64  
 1   city     275616 non-null  object 
 2   country  275616 non-null  object 
 3   lat      275616 non-null  float64
 4   long     275616 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 10.5+ MB


In [34]:
df_users.describe()

,user_id,lat,long
count,275616.000000,275616.000000,275616.000000
mean,499245.801521,37.111680,-93.981772
std,288533.458944,5.209627,18.086486
min,3.000000,19.700000,-157.800000
25%,249544.500000,33.660000,-112.200000
50%,498861.000000,37.740000,-88.930000
75%,748856.500000,40.700000,-78.910000
max,999999.000000,61.180000,30.310000


In [35]:
# Count NA values

df_users_missing_cts = pd.DataFrame({#'column_name': df_test_res.columns,
                                 'missing cts': df_users.isnull().sum()})
print('Missing value counts')
df_users_missing_cts.sort_values(by=['missing cts'], ascending=False)

Missing value counts


,missing cts
user_id,0
city,0
country,0
lat,0
long,0


In [36]:
# Count unique values for city and country

df_users[['city','country']].nunique()

city       923
country      1
dtype: int64

In [37]:
df_users.head()

,user_id,city,country,lat,long
0,510335,Peabody,USA,42.53,-70.97
1,89568,Reno,USA,39.54,-119.82
2,434134,Rialto,USA,34.11,-117.39
3,289769,Carson City,USA,39.15,-119.74
4,939586,Chicago,USA,41.84,-87.68


In [38]:
# Check for duplicate user ids
# This would indicate if we have the same user multiple times
# If True, we have all unique ids, if false, there are duplicates

df_users['user_id'].nunique() == len(df_users)

True

In [39]:
# Check if there are differences in number of user ids between test and user df
# True indicates same number of users, False indicates different numbers

df_test_res['user_id'].nunique() == df_users['user_id'].nunique()

False

In [40]:
# Print number of unique users in both df

print(f"Test data number of unique users: {df_test_res['user_id'].nunique()}")
print(f"User data number of unique users: {df_users['user_id'].nunique()}")

Test data number of unique users: 316800
User data number of unique users: 275616


## Export clean dataset

In [41]:
# Inner join test data and user data

df_clean = pd.merge(left=df_test_res, right=df_users, on='user_id',
                    how='inner')

In [42]:
# Export csv of clean joined data

df_clean.to_csv(rpath+'data/clean/data_clean.csv')